# Executive summary

I use the jargon $S, T, U, V$ from `companion-solver-implementation-reference.pdf`

1. something spooky: quadratic formula seems to work 54% of the time, while one of the coefficients is > 0 54% of the time. 
2. ok beyond the 54% coincidence, it's weird that the quadratic formula isn't working
3. $S$ and $T$ are strongly anticorrelated. This could be bad for linear algebra.
4. $U$ and $V$ are `-0.434316` anticorrelated, thank god.

In [102]:
from functools import reduce, partial
product = partial(reduce, lambda x,y: x * y)

import jax.numpy as jnp
from jax import random, vmap, grad

import arviz as az
from matplotlib import pyplot as plt
from ipywidgets.widgets import FloatSlider
from ipywidgets import interact
from scipy.interpolate import BSpline
from scipy.stats import gaussian_kde, zscore

import numpyro
import numpyro.distributions as dist
import numpyro.optim as optim
from numpyro.diagnostics import hpdi, print_summary
from numpyro.infer import Predictive, SVI, Trace_ELBO, init_to_value, NUTS, MCMC
from numpyro.infer.autoguide import AutoLaplaceApproximation, AutoNormal, AutoDiagonalNormal

import xgboost as xgb

import pandas as pd

from report import Report

key = random.PRNGKey(0)

In [103]:

reportD = Report("data/d-10262021.csv", abso=False, unknown_feature="D")
reportB0 = Report("data/b0-10262021.csv", abso=False, unknown_feature="B0")
reportB1 = Report("data/b1-10262021.csv", abso=False, unknown_feature="B1")
reportB2 = Report("data/b2-10262021.csv", abso=False, unknown_feature="B2")


In [104]:
# doing something where the root doesn't matter
df = pd.concat((reportD.df, reportB0.df, reportB1.df, reportB2.df)).drop(["Root", "newton_delta", "squared_newton_delta", "Y"], axis=1)

In [105]:
# see companion-solver-implementation-reference.pdf
# I_k = xk^2 + V x + U
def U_0(A, D, B1, B2):
    return - D ** (3 + 1) / A / 3 ** (2 * 3) / B1 / B2
def U_1(A, D, B0, B2): 
    return - D ** (3 + 1) / A / 3 ** (2 * 3) / B0 / B2
def U_2(A, D, B0, B1): 
    return - D ** (3 + 1) / A / 3 ** (2 * 3) / B0 / B1

def V_0(A, D, B1, B2): 
    return B1 + B2 + (1 / A / 3 ** 3 - 1) * D
def V_1(A, D, B0, B2): 
    return B0 + B2 + (1 / A / 3 ** 3 - 1) * D
def V_2(A, D, B0, B1): 
    return B0 + B1 + (1 / A / 3 ** 3 - 1) * D

In [106]:
(U_0(85, df.D, df.B1, df.B2) > 0).sum() / df.shape[0], (U_1(85, df.D, df.B0, df.B2) > 0).sum() / df.shape[0], (U_2(85, df.D, df.B0, df.B1) > 0).sum() / df.shape[0]

(0.0, 0.0, 0.0)

In [107]:
# V is positive 50% of the time, well when you make `A` variable drawn from the data they each go exactly to 54 which is weird.
(V_0(df.A, df.D, df.B1, df.B2) > 0).sum() / df.shape[0], (V_1(df.A, df.D, df.B0, df.B2) > 0).sum() / df.shape[0], (V_2(df.A, df.D, df.B0, df.B1) > 0).sum() / df.shape[0]

(0.5464, 0.542625, 0.544925)

In [108]:
# quadratic formula
def discriminant(V, U): 
    return V ** 2 - 4 * 1 * U
def left(V): 
    return - V / 2 / 1

def left_root(V, U): 
    return left(V) - discriminant(V, U) ** (1 / 2) / 2
def right_root(V, U): 
    return left(V) + discriminant(V, U) ** (1 / 2) / 2

left_root(V_0(85, df.D, df.B1, df.B2), U_0(85, df.D, df.B1, df.B2)).mean(), right_root(V_0(85, df.D, df.B1, df.B2), U_0(85, df.D, df.B1, df.B2)).mean()

(-75082024999.88412, 74462246769.95874)

In [138]:
V = V_0(df.A, df.D, df.B1, df.B2)
U = U_0(df.A, df.D, df.B1, df.B2)
root = right_root(V, U)
(df.assign(should_be_zero=root ** 2 + V * root + U)[df.V_0 < 0].should_be_zero < 1e-7).sum() / df[df.V_0 < 1e-7].shape[0] #  < 1e-7).sum() / df.shape[0]# :( should be 100

0.5597442680776014

In [139]:
df.assign(should_be_zero=root ** 2 + V * root + U)

,A,D,B0,B1,B2,S,T,V_0,U_0,should_be_zero
0,1.225385,1.325426e+11,5.755128e+10,6.542221e+10,1.281316e+11,-7.506518e+48,2.990695e+37,6.501732e+10,-4.121358e+19,0.0
1,1.161466,9.382895e+10,6.780278e+10,1.414878e+11,1.503699e+11,-3.214892e+49,8.942582e+37,2.010208e+11,-4.302626e+18,757760.0
2,1.181081,2.372545e+11,4.180750e+10,4.228169e+09,1.923611e+11,-5.023085e+47,2.107946e+36,-3.322526e+10,-4.524607e+21,-524288.0
3,3.741670,1.073810e+11,1.916746e+11,1.893570e+11,1.669744e+11,-2.057915e+50,3.756915e+38,2.500133e+11,-1.541649e+18,-1418240.0
4,2.121444,2.484188e+11,2.007496e+11,5.616816e+10,3.753127e+10,-7.721380e+48,2.623457e+37,-1.503824e+11,-1.168142e+21,1310720.0
...,...,...,...,...,...,...,...,...,...,...
9995,2.525826,1.547166e+11,1.334635e+11,1.235127e+11,2.334470e+11,-1.169444e+50,2.385601e+38,2.045117e+11,-1.079237e+19,415744.0
9996,3.653421,4.243480e+11,6.425476e+10,1.457139e+11,1.425286e+11,-2.914812e+49,8.272637e+37,-1.318036e+11,-5.862171e+20,-131072.0
9997,0.019967,1.415918e+11,1.398386e+10,3.787796e+09,4.011951e+10,-7.623037e+45,1.317361e+35,1.649611e+11,-1.817098e+23,-33554432.0
9998,1.013678,5.817431e+10,1.399639e+11,2.703381e+10,5.402211e+10,-2.799449e+48,1.267157e+37,2.500714e+10,-1.061252e+19,14336.0


In [140]:
root

0       6.278237e+08
1       2.140161e+07
2       8.589889e+10
3       6.166116e+06
4       1.577858e+11
            ...     
9995    5.275778e+07
9996    1.361106e+11
9997    3.517001e+11
9998    4.174125e+08
9999    3.914614e+10
Length: 40000, dtype: float64

In [110]:
# I_D = D ^ {n + 1} + T * D + S
def S(A, *balances): 
    n = len(balances)
    return - A * n ** (2 * n) * product(balances) * sum(balances)

def T(A, *balances): 
    n = len(balances)
    return (A + 1 / n ** n) * n ** (2 * n) * product(balances)

In [119]:
S_A85 = S(85, df.B0, df.B1, df.B2)
T_A85 = T(85, df.B0, df.B1, df.B2)
df = df.assign(S=S_A85, T=T_A85)
downsampled = df.sample(frac=0.1)
df[["S", "T"]].corr()

,S,T
S,1.000000,-0.980513
T,-0.980513,1.000000


In [120]:
df = df.assign(V_0=V, U_0=U)

In [121]:
df[["V_0", "U_0"]].corr()

,V_0,U_0
V_0,1.000000,-0.434316
U_0,-0.434316,1.000000


In [130]:
(discriminant(V, U) < 1e-3).sum() # complex numbers are not the reason I broke the quadratic formula. But it's nice to see that 

0

In [132]:
discriminant(V, U)

0       4.392107e+21
1       4.042655e+22
2       1.920234e+22
3       6.251281e+22
4       2.728744e+22
            ...     
9995    4.186821e+22
9996    1.971707e+22
9997    7.540513e+23
9998    6.678071e+20
9999    2.420919e+21
Length: 40000, dtype: float64

In [145]:
df.B2.mean()

115039925432.79308